In [1]:
!pip install accelerate quanto transformers peft

In [2]:
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/


In [3]:
import pandas as pd
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline, AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from abc import ABC, abstractmethod
import os
from tqdm import tqdm

In [5]:
class TextGenerator(ABC):
    """
    Abstract base class for text generation models.
    """

    @abstractmethod
    def generate(self, prompt: str, max_length: int = 256) -> str:
        """
        Generate text based on the input prompt.

        Args:
        prompt (str): The input text prompt to generate text from.
        max_length (int): The maximum length of the generated text.

        Returns:
        str: The generated text.
        """
        pass

In [34]:
class QuestionGenerator():
    def __init__(self,
                     model: TextGenerator,
                     prompt: str = """Ti forniró un oggetto e  una relazione. Genera una domanda. Nella generazione delle domande
                     attieniti il piú possibile al oggetto e alla relazione forniti. Produci in output solo la domanda, non la risposta.

Oggetto: "Sagrada Familia"
Relazione: "architetto"
Domanda: Chi è l'architetto della Sagrada Familia?

Oggetto: {object_target}
Relazione:{relationship}
"""
                     ):
      self.model = model
      self.prompt = prompt
      pass

    def generate(self,
                 object_target,
                 relationship,
                 answer1,
                 answer2,
                 answer3,
                 answer4
                ):
        """
        Generate text based on the input prompt.

        Args:
        object_target (str): the subject of the question
        relationship (str): the relationshio of the question
        answer1 (str): one of the answer
        answer2 (str): one of the answer
        answer3 (str): one of the answer
        answer4 (str): one of the answer

        Returns:
        str: The final prompt of the LLM
        """

        answers = """A) {answer1}
B) {answer2}
C) {answer3}
D) {answer4}""".format(
            answer1=answer1,
            answer2=answer2,
            answer3=answer3,
            answer4=answer4
          )

        output_model = self.model.generate(self.prompt.format(
            object_target=object_target,
            relationship=relationship
            ))

        return  output_model + "\n" +answers

In [6]:
class FaunoModel(TextGenerator):
    def __init__(self, device: str = "cpu"):
        self.tokenizer = LlamaTokenizer.from_pretrained("baffo32/decapoda-research-llama-7B-hf")
        self.model = LlamaForCausalLM.from_pretrained(
            "baffo32/decapoda-research-llama-7B-hf",
            load_in_8bit=True,
            device_map=device
        )
        self.model = PeftModel.from_pretrained(self.model, "andreabac3/Open_Fauno-Italian-LLM-7bB")
        self.model.eval()

    def generate(self, question, max_length=256):
        prompt = f"The conversation between human and AI assistant.\n[|Human|] {question}.\n[|AI|] "
        inputs = self.tokenizer(prompt, return_tensors="pt")
        input_ids = inputs["input_ids"].cuda()
        generation_output = self.model.generate(
            input_ids=input_ids,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=256
        )
        output = self.tokenizer.decode(generation_output.sequences[0]).split("[|AI|]")[1]
        return output

In [7]:
class LLaMantinoModel(TextGenerator):
    def __init__(self,
                 device: str = "cuda",
                 model_id: str = "swap-uniba/LLaMAntino-2-7b-hf-dolly-ITA",
                 quantization : str = "float8"
                ):
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.tokenizer.add_special_tokens({"pad_token":"<unk>"})
        self.tokenizer.chat_template =   "{% set ns = namespace(i=0) %}" \
                                    "{% for message in messages %}" \
                                        "{% if message['role'] == 'user' and ns.i == 0 %}" \
                                               "{{ bos_token +' [INST] <<SYS>>\n' }}" \
                                               "{{ 'Sei un assistente disponibile, rispettoso e onesto di nome Llamantino. ' }}" \
                                               "{{ 'Rispondi sempre nel modo più utile possibile, pur essendo sicuro. ' }}" \
                                               "{{ 'Le risposte non devono includere contenuti dannosi, non etici, razzisti, sessisti, tossici, pericolosi o illegali. ' }}" \
                                               "{{ 'Assicurati che le tue risposte siano socialmente imparziali e positive. ' }}" \
                                               "{{ 'Se una domanda non ha senso o non è coerente con i fatti, spiegane il motivo invece di rispondere in modo non corretto. ' }}" \
                                               "{{ 'Se non conosci la risposta a una domanda, non condividere informazioni false.\n' }}" \
                                               "{{ '<</SYS>>\n\n' }}" \
                                               "{{ message['content'] + ' [/INST]' }}" \
                                        "{% elif message['role'] == 'user' and ns.i != 0 %} " \
                                            "{{ bos_token + ' [INST] ' + message['content'] + ' [/INST]' }}" \
                                        "{% elif message['role'] == 'assistant' %}" \
                                            "{{ ' '  + message['content'] + ' ' + eos_token + ' ' }}" \
                                        "{% endif %}" \
                                        "{% set ns.i = ns.i+1 %}" \
                                    "{% endfor %}"

        #quantization_config = QuantoConfig(weights=quantization)
        self.model = AutoModelForCausalLM.from_pretrained(model_id,
                                                          device_map=device,
                                                          load_in_8bit=True,
                                                          #quantization_config=quantization_config
                                                          )
        self.model.eval()

        self.pipe = pipeline(model=self.model,
            device_map="balanced",
            tokenizer=self.tokenizer,
            return_full_text=False,  # langchain expects the full text
            task='text-generation',
            max_new_tokens=512,  # max number of tokens to generate in the output
            temperature=0.7 #temperature
        )

    def generate(self, prompt, max_length=256):
        messages = [{"role": "user", "content": prompt}]
        text = self.tokenizer.apply_chat_template(messages, tokenize=False)

        sequences = self.pipe(text)
        output = ""
        for seq in sequences:
            output += output + seq['generated_text']
        return output

## Data Loading

In [8]:
dataset = pd.read_csv("datasetItaliano.csv",)
dataset

,Object,Relationship,Correct,Answer_1,Answer_2,Answer_3,Answer_4
0,Adelaide,unità amministrativa in cui è situato,Australia Meridionale,Taiwan,Australia Meridionale,Mosca,Messico
1,Albania,relazione diplomatica,Francia,Grecia,Francia,Canada,Brasile
2,Angola,relazione diplomatica,Australia,Australia,Canada,Italia,Regno Unito
3,Australia,relazione diplomatica,Birmania,Indonesia,Australia,Birmania,Georgia
4,Australia Meridionale,Paese,Australia,Grecia,Danimarca,Austria,Australia
...,...,...,...,...,...,...,...
95,United States Marine Corps,luogo di fondazione,Filadelfia,Indonesia,Russia,Filadelfia,Taiwan
96,Vietnam,relazione diplomatica,Ucraina,Turchia,Cina,Ucraina,Canada
97,Virginia,Paese,Stati Uniti d'America,Iran,Danimarca,Indonesia,Stati Uniti d'America
98,Virginia,confina con,Carolina del Nord,Turchia,Germania,Taiwan,Carolina del Nord


# Text Generation

In [9]:
#fauno7b = FaunoModel(device = "cuda")

In [10]:
#fauno7b.generate("Qual'è il significato della vita?")

In [11]:
#question = "Ti forniró delle triple composte da 2 oggetti e una relazione. Genera una domanda riguarda la relazione. " + "Esempio: oggetto1 = Romolo relazione = Fondatore oggetto2 = Roma" + " Domanda: Romolo é il fondatore di Roma?" + " oggetto1 = Beaudine relazione=regista oggetto2= 'Road to Paradise'"

#prompt = f"The conversation between human and AI assistant.\n[|Human|] {question}.\n[|AI|] "
#inputs = fauno7b.tokenizer(prompt, return_tensors="pt")
#input_ids = inputs["input_ids"].cuda()
#generation_output = fauno7b.model.generate(
#            input_ids=input_ids,
#            return_dict_in_generate=True,
#            output_scores=True,
#            max_new_tokens=256
#        )
#fauno7b.tokenizer.decode(generation_output.sequences[0]).split("[|AI|]")

In [12]:
#fauno7b.generate("Ti forniró delle triple composte da 2 oggetti e una relazione. Genera una domanda riguarda la relazione." +
#                      "Esempio: oggetto1 = Romolo relazione = Fondatore oggetto2 = Roma" +
#                      "Domanda: Romolo é il fondatore di Roma?" +
#                       "oggetto1 = Beaudine relazione=regista oggetto2= 'Road to Paradise'" )

In [13]:
model_id = "swap-uniba/LLaMAntino-2-chat-13b-hf-UltraChat-ITA"
llamantino13b = LLaMantinoModel(model_id = model_id, device = "cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

In [14]:
llamantino13b.generate("Qual'è il significato della vita?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


" Non ho credenze personali o opinioni. Tuttavia, il significato della vita è un argomento complesso e soggettivo che varia da persona a persona. alcuni possono credere che il significato della vita sia trovare la felicità, raggiungere i propri obiettivi, o contribuire al mondo in qualche modo. altri possono credere che il significato della vita sia trovare un senso di scopo, di connessione, o di significato in un'esperienza spirituale o religiosa. in definitiva, il significato della vita è una questione personale che può essere plasmata da esperienze, valori e credenze individuali."

In [15]:
llamantino13b.generate("Sei in grado di formulare delle domande?")

' Sì, come modello di lingua AI, posso formulare domande. Posso anche rispondere a domande e fornire informazioni su vari argomenti.'

In [16]:
llamantino13b.generate("Ti forniró delle triple composte da 2 oggetti e una relazione. Genera una domanda riguarda la relazione." +
                      " Esempio: oggetto1 = Romolo relazione = Fondatore oggetto2 = Roma" +
                      " Domanda: Romolo é il fondatore di Roma?" +
                      "oggetto1 = Beaudine relazione=regista oggetto2= 'Road to Paradise'" )

" Sì, posso fornirvi una triple composta da 2 oggetti e una relazione.\n\nOggetto1 = Beaudine relazione = Regista oggetto2 = 'Road to Paradise'\n\nLa domanda è: Beaudine è il regista di 'Road to Paradise'?"

In [17]:
llamantino13b.generate("""Ti forniró delle triple composte da un oggetto,  una relazione e quattro possibili risposte. Genera una domanda a scelta multipla.
 Esempio: oggetto1 =  libro '1984'  relazione = autore risposta1= Huxley risposta2 = Orwell risposta3 = Hemingway risposta4 = Bradbury
 Domanda: Chi ha scritto il libro '1984'?
A) Huxley
B) Orwell
C) Hemingway
D) Bradbury

 oggetto1 = 'Road to Paradise' relazione=regista risposta1= Beaudine risposta2 = Scorsese risposta3 =  Spielberg risposta4 = Nolan""")

" Domanda: Chi ha diretto il film 'Road to Paradise'?\nA) Beaudine\nB) Scorsese\nC) Spielberg\nD) Nolan"

In [37]:
questionGenerator = QuestionGenerator(model = llamantino13b)

question = questionGenerator.generate(
            object_target="Germania",
            relationship="confine",
            answer1="Irlanda",
            answer2="Belgio",
            answer3="Portogallo",
            answer4="Bulgaria"
)

question

' Domanda: Quale paese ha il confine più lungo con la Germania?\nA) Irlanda\nB) Belgio\nC) Portogallo\nD) Bulgaria'

In [35]:
# Path to the output file
output_file = 'questions.txt'

# Check if file exists, if not, create it by opening and closing it immediately
if not os.path.exists(output_file):
    open(output_file, 'w').close()

In [38]:
# Now, append data to the file
with open(output_file, 'a', encoding='utf-8') as file:
    for index, row in tqdm(dataset.iterrows(), total=dataset.shape[0], desc="Writing rows"):
      #Formatting the data into a string
      question = questionGenerator.generate(
            object_target=row["Object"],
            relationship=row["Relationship"],
            answer1=row["Answer_1"],
            answer2=row["Answer_2"],
            answer3=row["Answer_3"],
            answer4=row["Answer_4"]
      )

      # Append the result to the file
      file.write(question + '\tCorrect:'+ row["Correct"] + '\n')
      #print(question + '\t'+ row["Correct"] + '\n')

Writing rows: 100%|██████████| 100/100 [09:34<00:00,  5.75s/it]
